In [1]:
!pip install catboost

     |████████████████████████████████| 76.3 MB 1.6 MB/s 


In [56]:
import pandas as pd
from pandas.api.types import is_numeric_dtype
from sklearn.model_selection import train_test_split
import catboost as cb
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv('/content/titanic.csv')

In [58]:
df.dropna(subset=['Survived'],inplace=True)

In [59]:
X = df[['Pclass','Sex', 'Age', 'Fare']]
y = df['Survived']

In [60]:
X['Pclass'] = X['Pclass'].astype('str')
X['Fare'].fillna(0,inplace=True)
X['Age'].fillna(0,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [61]:
def get_categorical_indicies(X):
  cats = []
  for col in X.columns:
    if is_numeric_dtype(X[col]):
      pass
    else:
      cats.append(col)
  cat_indicies = []
  for col in cats:
    cat_indicies.append(X.columns.get_loc(col))
  return cat_indicies
categorical_indicies = get_categorical_indicies(X)

In [62]:
def convert_cats(X):
  cats = []
  for col in X.columns:
    if is_numeric_dtype(X[col]):
      pass
    else:
      cats.append(col)
    cat_indicies = []
    for col in cats:
      X[col] = X[col].astype('category')
convert_cats(X)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101, stratify=y)

In [64]:
print('Test Survival Rate:',y_test.sum()/y_test.count())

Test Survival Rate: 0.3854748603351955


In [65]:
train_df = pd.concat([X,y],axis=1)
survived = train_df[train_df['Survived']==1]
deceased = train_df[train_df['Survived']==0]
deceased = deceased.sample(n=len(survived), random_state=101)
train_df = pd.concat([survived,deceased],axis=0)
X_train = train_df.drop('Survived',axis=1)
y_train = train_df['Survived']

In [66]:
train_dataset = cb.Pool(X_train,y_train, cat_features=categorical_indicies)
test_dataset = cb.Pool(X_test,y_test, cat_features=categorical_indicies)

In [67]:
model = cb.CatBoostClassifier(loss_function='Logloss', eval_metric='Accuracy')

In [68]:
grid = {'learning_rate': [0.03, 0.1],
'depth': [4, 6, 10],
'l2_leaf_reg': [1, 3, 5,],
'iterations': [50, 100, 150]}

In [69]:
model.grid_search(grid,train_dataset)

Streaming output truncated to the last 5000 lines.
86:	learn: 0.8062157	test: 0.7445255	best: 0.7810219 (1)	total: 117ms	remaining: 17.5ms
87:	learn: 0.8062157	test: 0.7299270	best: 0.7810219 (1)	total: 118ms	remaining: 16.1ms
88:	learn: 0.8043876	test: 0.7299270	best: 0.7810219 (1)	total: 119ms	remaining: 14.7ms
89:	learn: 0.8025594	test: 0.7299270	best: 0.7810219 (1)	total: 120ms	remaining: 13.4ms
90:	learn: 0.8043876	test: 0.7299270	best: 0.7810219 (1)	total: 122ms	remaining: 12ms
91:	learn: 0.8025594	test: 0.7299270	best: 0.7810219 (1)	total: 123ms	remaining: 10.7ms
92:	learn: 0.8025594	test: 0.7445255	best: 0.7810219 (1)	total: 124ms	remaining: 9.35ms
93:	learn: 0.8007313	test: 0.7445255	best: 0.7810219 (1)	total: 126ms	remaining: 8.02ms
94:	learn: 0.8007313	test: 0.7445255	best: 0.7810219 (1)	total: 127ms	remaining: 6.67ms
95:	learn: 0.8062157	test: 0.7299270	best: 0.7810219 (1)	total: 128ms	remaining: 5.33ms
96:	learn: 0.8062157	test: 0.7299270	best: 0.7810219 (1)	total: 129ms	r

{'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,
               46,
               47,
               48,
               49],
            

In [70]:
model.get_params()

{'depth': 10,
 'eval_metric': 'Accuracy',
 'iterations': 50,
 'l2_leaf_reg': 1,
 'learning_rate': 0.03,
 'loss_function': 'Logloss'}

In [71]:
pred = model.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.77      0.89      0.82       110
           1       0.76      0.57      0.65        69

    accuracy                           0.77       179
   macro avg       0.77      0.73      0.74       179
weighted avg       0.77      0.77      0.76       179



In [89]:
print(model.get_best_score())

{'learn': {'Accuracy': 0.7850877192982456, 'Logloss': 0.4767093628366711}}


In [81]:
model.calc_feature_statistics(test_dataset,feature='Sex',plot=True,prediction_type='Class')

{'binarized_feature': array([1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0,
        1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0,
        1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1,
        1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0,
        0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1,
        1, 1, 0]),
 'cat_values': array(['female', 'male'], dtype='<U6'),
 'mean_prediction': array([0.97959185, 0.02307692], dtype=float32),
 'mean_target': array([0.7346939 , 0.25384617], dtype=float32),
 'mean_weighted_target': array([], dtype=float32),
 'objects_per_bin': array([ 49, 130], dtype=uint32),
 'predictions_on_varying_feature': array([0.98324022, 0.        ])}